# Wavelet Transform + CNN Model



In [7]:
import os
import pandas as pd
import numpy as np
from obspy import read
from tqdm import tqdm
import scipy.stats as stats

# Paths
augmented_data_path = '/mnt/c/Users/Usuario/Documents/Studies/GicoProject/SeismicWaves/data/procesed/used_data/training_augmented'
features_path = '/mnt/c/Users/Usuario/Documents/Studies/GicoProject/SeismicWaves/data/procesed/features'

# Create features directory if it doesn't exist
if not os.path.exists(features_path):
    os.makedirs(features_path)

In [8]:
def extract_wavelet_features(signal, wavelet='db4', level=4):
    """Extract statistical features from wavelet decomposition of a signal.
    Args:
        signal: Input signal array
        wavelet: Wavelet type to use
        level: Decomposition level
    Returns:
        array: Feature vector containing statistical measures"""
    # Perform wavelet decomposition
    coeffs = pywt.wavedec(signal, wavelet, level=level)
    
    # Initialize feature list
    features = []
    
    # Extract features from each coefficient level
    for coef in coeffs:
        # Statistical features
        features.extend([
            np.mean(coef),           # Mean
            np.std(coef),            # Standard deviation
            stats.skew(coef),        # Skewness
            stats.kurtosis(coef),    # Kurtosis
            np.percentile(coef, 75), # 75th percentile
            np.percentile(coef, 25), # 25th percentile
            np.max(coef),            # Maximum
            np.min(coef),            # Minimum
            np.sum(np.abs(coef)),    # L1 norm
            np.sqrt(np.sum(coef**2)),# L2 norm
            stats.entropy(np.abs(coef)), # Signal entropy
            np.median(np.abs(coef))  # Median absolute deviation
        ])
        
    return np.array(features)

def process_seismic_files(data_path, arrival_times_csv):
    """Process all seismic files and extract wavelet features.
    Args:
        data_path: Path to directory containing MSEED files
        arrival_times_csv: Path to CSV with arrival times
    Returns:
        tuple: (features array, arrival times array, file names)"""
    # Read arrival times
    arrivals_df = pd.read_csv(arrival_times_csv)
    
    features_list = []
    arrival_times = []
    file_names = []
    
    print('Extracting wavelet features...')
    for _, row in tqdm(arrivals_df.iterrows(), total=len(arrivals_df)):
        file_path = os.path.join(data_path, row['augmented_file'])
        
        try:
            # Read seismic signal
            st = read(file_path)
            signal = st[0].data
            
            # Extract features
            features = extract_wavelet_features(signal)
            features_list.append(features)
            arrival_times.append(row['arrival_time'])
            file_names.append(row['augmented_file'])
            
        except Exception as e:
            print(f'Error processing {file_path}: {str(e)}')
            continue
    
    return np.array(features_list), np.array(arrival_times), file_names

# Process all files
augmented_path = os.path.join(augmented_data_path, 'augmented')
arrival_times_csv = os.path.join(augmented_data_path, 'arrival_times.csv')

X, y, files = process_seismic_files(augmented_path, arrival_times_csv)

# Save features and metadata
np.save(os.path.join(features_path, 'wavelet_features.npy'), X)
np.save(os.path.join(features_path, 'arrival_times.npy'), y)
pd.DataFrame({'file': files}).to_csv(
    os.path.join(features_path, 'feature_files.csv'), index=False)

print(f'Extracted features shape: {X.shape}')
print(f'Number of samples: {len(files)}')

# Update the feature extraction info
print('Features extracted per coefficient level:', 12)
print('Total features for 4 levels:', 12 * (4 + 1))  # 4 detail + 1 approximation

Extracting wavelet features...


  0%|          | 0/4989 [00:00<?, ?it/s]

100%|██████████| 4989/4989 [03:34<00:00, 23.21it/s] 


Extracted features shape: (4989, 60)
Number of samples: 4989
Features extracted per coefficient level: 12
Total features for 4 levels: 60


In [11]:
# Show example of features for one file
example_idx = 0
print(f'Features for file {files[example_idx]}:')
print('Feature vector length:', len(X[example_idx]))
print('\nFirst 10 features:')
print(X[example_idx][:10])
print(f'\nArrival time: {y[example_idx]:.2f}s')

Features for file 01010056.mseed:
Feature vector length: 60

First 10 features:
[-2.26887150e-03  1.36387755e-01  2.11445763e+00  9.02215459e+01
  1.74969649e-02 -1.97850397e-02  1.76067501e+00 -1.31033293e+00
  1.88985862e+01  2.85151844e+00]

Arrival time: 30.60s


In [10]:
import numpy as np

# Cargar los archivos
features_path = '/mnt/c/Users/Usuario/Documents/Studies/GicoProject/SeismicWaves/data/procesed/features'
X = np.load(os.path.join(features_path, 'wavelet_features.npy'))
y = np.load(os.path.join(features_path, 'arrival_times.npy'))

# Verificar las dimensiones
print(f"Forma de X: {X.shape}")
print(f"Forma de y: {y.shape}")

# Ver los primeros elementos
print("\nPrimeros 5 elementos de X:")
print(X[:5])
print("\nPrimeros 5 elementos de y:")
print(y[:5])

Forma de X: (4989, 60)
Forma de y: (4989,)

Primeros 5 elementos de X:
[[-2.26887150e-03  1.36387755e-01  2.11445763e+00  9.02215459e+01
   1.74969649e-02 -1.97850397e-02  1.76067501e+00 -1.31033293e+00
   1.88985862e+01  2.85151844e+00  4.90902773e+00  1.87466164e-02
  -4.68282065e-03  4.36041154e-01 -1.00966551e+00  3.50142234e+01
   7.43629529e-02 -8.48785630e-02  3.57025540e+00 -3.56212458e+00
   7.44136070e+01  9.11576754e+00  5.07205141e+00  8.28264720e-02
   5.64923997e-02  1.62441522e+00  4.88074603e+00  1.10670185e+02
   2.23716437e-01 -2.32970680e-01  2.68300654e+01 -1.51858868e+01
   4.54496807e+02  4.78871929e+01  5.62304109e+00  2.28371860e-01
   3.33488387e-05  1.16491400e+00 -5.91348512e-01  7.28446100e+01
   1.35390393e-01 -1.25288355e-01  1.26437588e+01 -1.69246391e+01
   6.01567335e+02  4.84525808e+01  6.10403926e+00  1.28995578e-01
  -1.18978216e-05  1.56823783e-01 -9.33989454e-01  7.02097351e+01
   2.04853454e-02 -1.97404723e-02  1.95495840e+00 -2.39828782e+00
   1.

## Preprocesamiento de señales

Para entrenar una CNN, todas las señales de entrada deben tener la misma longitud. Hay tres casos posibles:

1. La señal tiene exactamente 3000 muestras → se usa tal cual
2. La señal tiene más de 3000 muestras → se corta al tamaño deseado
3. La señal tiene menos de 3000 muestras → se rellena con ceros hasta alcanzar 3000

Esto asegura que todas las señales tengan dimensiones consistentes para el entrenamiento.

## Carga de datos preprocesados

Las señales sísmicas ya han sido preprocesadas con:
1. Normalización Z-score
2. Filtro pasabanda (bandpass filter)

Por lo tanto, usaremos las señales directamente sin procesamiento adicional.

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Cargar datos de wavelets y arrival times
X_wavelets = np.load(os.path.join(features_path, 'wavelet_features.npy'))
y = np.load(os.path.join(features_path, 'arrival_times.npy'))

# Cargar señales preprocesadas
raw_signals = []
files_df = pd.read_csv(os.path.join(features_path, 'feature_files.csv'))

print('Cargando señales preprocesadas...')
for file in tqdm(files_df['file']):
    file_path = os.path.join(augmented_data_path, 'augmented', file)
    st = read(file_path)
    signal = st[0].data
    raw_signals.append(signal)

X_raw = np.array(raw_signals)
# Ajustar dimensiones para CNN
X_raw = X_raw.reshape(X_raw.shape[0], -1, 1)

print(f'Forma de las señales crudas: {X_raw.shape}')
print(f'Forma de características wavelets: {X_wavelets.shape}')

# Dividir los datos
X_raw_train, X_raw_test, X_wav_train, X_wav_test, y_train, y_test = train_test_split(
    X_raw, X_wavelets, y, test_size=0.2, random_state=42
)

In [ ]:
# Crear el modelo híbrido
# Entrada para señales crudas
raw_input = tf.keras.layers.Input(shape=(3000, 1))
x1 = tf.keras.layers.Conv1D(32, 5, activation='relu')(raw_input)
x1 = tf.keras.layers.MaxPooling1D(2)(x1)
x1 = tf.keras.layers.Conv1D(64, 5, activation='relu')(x1)
x1 = tf.keras.layers.MaxPooling1D(2)(x1)
x1 = tf.keras.layers.Flatten()(x1)

# Entrada para características wavelets
wavelet_input = tf.keras.layers.Input(shape=(X_wavelets.shape[1],))
x2 = tf.keras.layers.Dense(64, activation='relu')(wavelet_input)
x2 = tf.keras.layers.Dropout(0.3)(x2)

# Combinar ambas entradas
combined = tf.keras.layers.concatenate([x1, x2])
x = tf.keras.layers.Dense(128, activation='relu')(combined)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
output = tf.keras.layers.Dense(1)(x)

model = tf.keras.Model(inputs=[raw_input, wavelet_input], outputs=output)

# Compilar el modelo
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

In [ ]:
# Entrenar el modelo
history = model.fit(
    [X_raw_train, X_wav_train],
    y_train,
    validation_data=([X_raw_test, X_wav_test], y_test),
    epochs=50,
    batch_size=32,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
    ]
)

In [ ]:
# Visualizar el entrenamiento
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('Model MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()

plt.tight_layout()
plt.show()

# Evaluar el modelo
test_loss, test_mae = model.evaluate([X_raw_test, X_wav_test], y_test)
print(f'\nTest Loss: {test_loss:.4f}')
print(f'Test MAE: {test_mae:.4f}')